# Applying Cloud Masks to a Batch of Images
### ECOSTRESS Tutorials
###### This code is best suited for use when you have a folder of files and their associated cloud masks that you would like to be applied. 
###### This code is written to cloud mask a Land Surface Temperature (LST) file, but may be modified for use with other ECOSTRESS data products.

### Import the Libraries we Need to Apply the Cloud Mask

In [ ]:
import os
from os import makedirs
from os.path import join, basename, splitext
from glob import glob
from datetime import datetime
import numpy as np
import pandas as pd
import rioxarray
import rasterio

### Set Constants for the Project

In [ ]:
#Replace this with the path to the folder where the downloaded files are kept, wrapped in quotes
input_directory = r"Replace_this_text_with_folder_path"
#Replace this with the path to the folder where you want the output files to be stored, wrapped in quotes
output_directory = r"Replace_this_text_with_folder_path"
#This line makes sure that the output directory exists
makedirs(output_directory, exist_ok=True)

#Establish aesthetics 
pd.set_option("display.max_colwidth", None) #Shows the entire table when it prints
alpha = 0.7 #Sets the transparency of the image to 70%
fig_width_px = 1080 #Defines the size of the figure
fig_height_px = 720
ET_cmap = [ #Lists the colors we want to use in our displays
    "#f6e8c3",
    "#d8b365",
    "#99974a",
    "#53792d",
    "#6bdfd2",
    "#1839c5"
]


### Collect File Names

In [ ]:
#Find all files in the input directory containing 'LST' and put them in a table
ST_raw_filenames = pd.DataFrame({"ST_raw_filename": sorted(glob(join(input_directory, "*_LST_*.tif")))})
#Add the date of the LST images from the file name to the table
ST_raw_filenames["datetime_UTC"] = ST_raw_filenames.ST_raw_filename.apply(lambda ST_raw_filename: datetime.strptime(splitext(basename(ST_raw_filename))[0].split("_")[-2][3:], "%Y%j%H%M%S"))
#Find all files in the input directory containing 'CloudMask' and put them in a table
cloud_filenames = pd.DataFrame({"cloud_filename": sorted(glob(join(input_directory, "*_CloudMask_*.tif")))})
#Add the date of the CloudMask images from the file name to the table
cloud_filenames["datetime_UTC"] = cloud_filenames.cloud_filename.apply(lambda cloud_filename: datetime.strptime(splitext(basename(cloud_filename))[0].split("_")[-2][3:], "%Y%j%H%M%S"))
#Merge the LST and CloudMask tables together
ST_filenames = pd.merge(ST_raw_filenames, cloud_filenames)
#Create the output file name 
ST_filenames["ST_masked_filename"] = ST_filenames.datetime_UTC.apply(lambda datetime_UTC: join(output_directory, f"{datetime_UTC:%Y.%m.%d.%H.%M.%S}_ST.tif"))
#View the table we created
ST_filenames = ST_filenames[["datetime_UTC", "ST_raw_filename", "cloud_filename", "ST_masked_filename"]]
ST_filenames

### Apply the Cloud Mask to the LST Image

In [ ]:
#Loop through each line of the table
for i, (datetime_UTC, ST_raw_filename, cloud_filename, ST_masked_filename) in ST_filenames.iterrows():
    print(f"ST masked file: {ST_masked_filename}")

    #Open the LST image file using rioxarray
    ST = rioxarray.open_rasterio(ST_raw_filename).squeeze("band", drop=True)
    #Convert 0s to NaN, then convert the values to Celsius
    ST.data = np.where(ST.data == 0, np.nan, ST.data*0.02)-273.15
    #Open the Cloud Mask image file using rioxarray and reproject it to be the same as the LST file
    cloud = rioxarray.open_rasterio(cloud_filename).squeeze("band", drop=True).rio.reproject_match(ST)
    #Bitshifts the array two bits to the right, and uses & 1 to create a binary cloud mask
    cloud.data = (cloud.data >> 2) & 1
    #Apply the Cloud Mask to the LST layer by saying if the cloud mask is true, set the pixel to NaN 
    ST.data = np.where(cloud.data, np.nan, ST.data)

    ###Remove any outliers
    #Calculate the 8% and 98% quantile
    low, high = np.nanquantile(ST.data, [0.08, 0.98])
    #Make any pixel lower than 8% quantile or higher than 98% quantile NaN
    ST.data = np.where((ST.data < low) | (ST.data > high), np.nan, ST.data)

    ##Make sure we are only using high quality scenes
    #Quantify how many pixels are missing (NaN)
    missing_proportion = np.count_nonzero(np.isnan(ST.data)) / ST.data.size
    #Print the number of pixels that are missing
    print(f"missing proportion: {missing_proportion * 100}%")

    #if more than 50% of pixels are missing, we will skip the image
    if missing_proportion > 0.5:
        print("Low quality scene. Skipping this file.")
        continue

    #Get the metadata from the original ST file
    with rasterio.open(ST_raw_filename) as src:
        out_meta = src.meta.copy()

    #Update the metadata without applying the scale factor again
    out_meta.update({
        "driver": "GTiff",
        "height": ST.shape[0],
        "width": ST.shape[1],
        "dtype": 'float32'
    })
    
    #Save the masked image with the updated metadata
    with rasterio.open(ST_masked_filename, 'w', **out_meta) as output_raster:
        output_raster.write(ST.data.astype('float32'), 1)